In [1]:
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
import os
import logging
from sklearn.model_selection import train_test_split

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
path_data_folder = Path.cwd().parent.parent / 'data'
print(path_data_folder)

folder_raw_data_milling = path_data_folder / "raw/milling"
folder_interim_data_milling = path_data_folder / "interim/milling"
folder_processed_data_milling = path_data_folder / "processed/milling"

/home/tim/Documents/feat-store/data


In [3]:
df_raw = pd.read_csv(folder_raw_data_milling / "milling.csv.gz", compression='gzip',)
df_raw.head()

,cut_id,cut_no,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class
0,0_0,0,1,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0
1,0_0,0,1,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0
2,0_0,0,1,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0
3,0_0,0,1,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0
4,0_0,0,1,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0


In [5]:
# add column for the y label, where 1 is the label for the positive class when the tool_class > 1
df_raw['y'] = df_raw['tool_class'] > 1
df_raw['y'] = df_raw['y'].astype(int)
df_raw.head()

,cut_id,cut_no,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class,y
0,0_0,0,1,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0,0
1,0_0,0,1,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0,0
2,0_0,0,1,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0,0
3,0_0,0,1,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0,0
4,0_0,0,1,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0,0


In [29]:
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)
df.head()

(11570, 4510)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [30]:
df = df[df['case']==15]

df['y'] = df['tool_class'] > 1
df['y'] = df['y'].astype(int)
print(df.shape)
df.head()

(520, 4511)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class,y
10654,154_0,3.277600e-05,3.138502e-07,-0.000004,0.002755,0.002738,0.002713,9.108564,0.103335,0.0,...,1.210316,9.495060,3.155575,1.471686,1.409514,4.198592,154,15,0,0
10655,154_1,-3.481216e-05,-4.725636e-05,-0.000036,0.002773,0.002761,0.002748,9.774098,0.106730,0.0,...,1.145610,6.663433,1.933154,1.306129,1.204335,1.378873,154,15,0,0
10656,154_10,-4.339720e-07,2.743999e-06,0.000019,0.002787,0.002761,0.002742,9.451535,0.103611,0.0,...,1.267046,11.902700,5.012953,4.047941,1.298777,8.912656,154,15,0,0
10657,154_11,1.703928e-05,3.375236e-05,0.000009,0.002836,0.002819,0.002831,10.165892,0.119084,0.0,...,1.365485,12.961481,6.541192,6.282037,1.916424,10.629192,154,15,0,0
10658,154_12,-8.043911e-06,-3.024958e-07,-0.000004,0.002694,0.002670,0.002650,8.906037,0.113346,0.0,...,1.257279,10.932990,4.016544,1.559468,1.285410,6.817448,154,15,0,0


In [31]:
# drop any columns with NaN values
df = df.dropna(axis=1, how='any')
df.set_index('cut_id', inplace=True)

print(df.shape)
df.head()

(520, 2803)


,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class,y
cut_id,,,,,,,,,,,,,,,,,,,,,
154_0,3.277600e-05,3.138502e-07,-0.000004,0.002755,0.002738,0.002713,9.108564,0.103335,0.0,1.0,...,1.210316,9.495060,3.155575,1.471686,1.409514,4.198592,154,15,0,0
154_1,-3.481216e-05,-4.725636e-05,-0.000036,0.002773,0.002761,0.002748,9.774098,0.106730,0.0,0.0,...,1.145610,6.663433,1.933154,1.306129,1.204335,1.378873,154,15,0,0
154_10,-4.339720e-07,2.743999e-06,0.000019,0.002787,0.002761,0.002742,9.451535,0.103611,0.0,1.0,...,1.267046,11.902700,5.012953,4.047941,1.298777,8.912656,154,15,0,0
154_11,1.703928e-05,3.375236e-05,0.000009,0.002836,0.002819,0.002831,10.165892,0.119084,0.0,1.0,...,1.365485,12.961481,6.541192,6.282037,1.916424,10.629192,154,15,0,0
154_12,-8.043911e-06,-3.024958e-07,-0.000004,0.002694,0.002670,0.002650,8.906037,0.113346,0.0,1.0,...,1.257279,10.932990,4.016544,1.559468,1.285410,6.817448,154,15,0,0


In [35]:
df = df.drop(['case', 'tool_class', 'y', 'cut_no'], axis=1)
df.head()

,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,smcac__permutation_entropy__dimension_4__tau_1,smcac__permutation_entropy__dimension_5__tau_1,smcac__permutation_entropy__dimension_6__tau_1,smcac__permutation_entropy__dimension_7__tau_1,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98"
cut_id,,,,,,,,,,,,,,,,,,,,,
154_0,3.277600e-05,3.138502e-07,-0.000004,0.002755,0.002738,0.002713,9.108564,0.103335,0.0,1.0,...,1.947155,2.458517,2.871755,3.184946,1.210316,9.495060,3.155575,1.471686,1.409514,4.198592
154_1,-3.481216e-05,-4.725636e-05,-0.000036,0.002773,0.002761,0.002748,9.774098,0.106730,0.0,0.0,...,2.074538,2.553512,2.975967,3.298287,1.145610,6.663433,1.933154,1.306129,1.204335,1.378873
154_10,-4.339720e-07,2.743999e-06,0.000019,0.002787,0.002761,0.002742,9.451535,0.103611,0.0,1.0,...,1.908081,2.360681,2.757294,3.136191,1.267046,11.902700,5.012953,4.047941,1.298777,8.912656
154_11,1.703928e-05,3.375236e-05,0.000009,0.002836,0.002819,0.002831,10.165892,0.119084,0.0,1.0,...,2.064300,2.490675,2.817898,3.131181,1.365485,12.961481,6.541192,6.282037,1.916424,10.629192
154_12,-8.043911e-06,-3.024958e-07,-0.000004,0.002694,0.002670,0.002650,8.906037,0.113346,0.0,1.0,...,2.091574,2.565142,2.939898,3.276163,1.257279,10.932990,4.016544,1.559468,1.285410,6.817448


In [32]:
dfy = df['y']
dfy.head()

cut_id
154_0     0
154_1     0
154_10    0
154_11    0
154_12    0
Name: y, dtype: int64

# Test Selection

In [71]:
from src.features.select_feat_from_dataframe import milling_select_features, milling_add_y_label_anomaly, milling_create_data_splits

In [70]:
# load first 500 rows of data from milling_features.csv.gz
# df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip', nrows=500)
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)
df.head()

(11570, 4510)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [80]:
df_train, df_val, df_test = milling_create_data_splits(df)

df_feat_train shape: (6942, 4510)
df_feat_test shape: (2314, 4510)
df_feat_val shape: (2314, 4510)


In [77]:
# count the number values for each 'tool_class' in df_train
df_train['tool_class'].value_counts()

1    3900
0    2536
2     506
Name: tool_class, dtype: int64

In [79]:
df_val['tool_class'].value_counts()

1    1300
0     845
2     169
Name: tool_class, dtype: int64

In [81]:
df_val[df_val['tool_class'] == 2]['case'].unique()

array([13, 11, 15, 10, 14,  5,  9])

In [69]:


df = milling_add_y_label_anomaly(df)
df = milling_select_features(df)


In [66]:
df.head()

,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class,y
cut_id,,,,,,,,,,,,,,,,,,,,,
50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,1.0,...,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1,0
50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,1.0,...,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1,0
50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,1.0,...,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1,0
50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,1.0,...,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1,0
50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,1.0,...,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1,0


# Split on Case Number

In [4]:
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)
df.head()

(11570, 4510)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [5]:
for i in df['case'].unique():
    print(i)


4
9
10
11
12
3
13
14
1
2
5
6
7
8
15
16


In [6]:
int(df[df['cut_no']==7]['tool_class'].unique())

1

In [7]:
# dictionary comprehension to create dict of each cut_no and its corresponding tool_class
cut_id_tool_class_dict = {i : int(df[df['cut_no']==i]['tool_class'].unique()) for i in df['cut_no'].unique()}

In [24]:
cut_numbers = []
tool_classes = []
for i in cut_id_tool_class_dict:
    cut_numbers.append(i)
    tool_classes.append(cut_id_tool_class_dict[i])


cut_numbers_train, cut_numbers_test, tool_classes_train, tool_classes_test = train_test_split(
    np.array(cut_numbers), np.array(tool_classes),  test_size=0.6, random_state=13, stratify=np.array(tool_classes)
)

In [25]:
print(len(cut_numbers_train))
cut_numbers_train

66


array([ 49, 107,  93, 154,  36,  60,  34,  52,  44,  55,  70,  82,  67,
        84, 120, 100, 144, 147,  15,   1,   3,  69,  64, 140,   2, 139,
        85,   7, 150,  63,  47,  61,  18, 119,  62,  81,  12,  78,  14,
        43, 157,  19,  13,   9, 110,  24, 145, 101,  87,  66,  91, 111,
       105,  77, 161,  35,  51,  21, 165, 138, 118,   4, 123,  26,  80,
        37])

In [28]:
np.intersect1d(cut_numbers_train, cut_numbers_test)

array([], dtype=int64)

In [67]:
from src.features.select_feat_from_dataframe import milling_stratify_cut_no

(cut_no_train, cut_no_val, cut_no_test) = milling_stratify_cut_no(df, random_seed=76, train_size_pct=0.40)


cut_numbers_train shape: 66
cut_numbers_test shape: 50
cut_numbers_val shape: 49


In [68]:
cut_no_train

array([162,  32,  46,  29, 105,  14,  91,  18,   9, 156,  15,  84,  35,
        85,  82,  87, 125,   6,  13,  73,  50,  42, 112,  86,  41, 100,
       153,  55,  27, 131, 140,  26,  59, 126,  88,  20, 154, 145,  30,
        31, 161, 147,   0,  48,  89,  72,  60,  33,  83, 110, 122,  34,
        92, 159,  45,  77, 143,  36,  19, 141,  47, 151, 116, 157, 149,
        51])

In [64]:
cut_no_train

array([  2,  69,  45,  72,  81, 165,  37, 141, 155, 131, 121,  40,  83,
       104,  23,  66,  97, 166, 154,  79, 112, 109,  44, 119, 117, 143,
        38, 161, 115,  43, 157, 106,  18,   0,  51,  63, 160,  91,  93,
        31,  82,  87, 162,  46,  55, 123,  21,  99,  27, 148,  12,  68,
        28, 108,  33,   9,  60,  77, 156,  92,  65,  59,  50, 122,  29,
        90])

In [36]:
print(len(cut_no_val))

49


In [34]:
cut_no_test

array([138,  12,  24, 102, 113, 104, 114, 158,  21,   7,  58,   4, 109,
       134,  63,  28,  23, 120,  93,  37,   3,  75, 163,  70,  99,  71,
       117, 128,  76,  52,  39, 129,  25,   2, 108,  69, 137, 124,  49,
        22, 135, 146,  67, 121,  74, 148, 119, 139, 103,  54])

# milling_select_features

In [75]:
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)

(11570, 4510)


In [80]:
df_feat_sel = milling_select_features(df, random_seed=76, train_size_pct=0.40)

cut_numbers_train shape: 66
cut_numbers_test shape: 50
cut_numbers_val shape: 49
df_train shape: (4674, 2798)
df_val shape: (3429, 2798)
df_test shape: (3467, 2798)


In [82]:
df_feat_sel.head()

,smcac__fourier_entropy__bins_2,ae_spindle__cid_ce__normalize_True,ae_spindle__partial_autocorrelation__lag_1,ae_spindle__autocorrelation__lag_1,smcac__fourier_entropy__bins_10,"smcdc__fft_coefficient__attr_""abs""__coeff_1",ae_spindle__autocorrelation__lag_2,"smcac__agg_autocorrelation__f_agg_""median""__maxlag_40","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",...,vib_table__binned_entropy__max_bins_10,"smcdc__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.2","ae_table__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0","ae_spindle__fft_aggregated__aggtype_""centroid""",ae_table__ar_coefficient__coeff_6__k_10,vib_spindle__large_standard_deviation__r_0.2,smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"ae_spindle__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""",ae_spindle__binned_entropy__max_bins_10,"ae_spindle__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0"
cut_id,,,,,,,,,,,,,,,,,,,,,
50_23,0.228632,7.401132,0.572352,0.572352,0.270641,4.147273,0.171175,0.022750,0.889350,0.740909,...,2.219080,0.000814,0.000638,3.475581,0.001809,1.0,0.078260,-0.004730,2.142089,0.000552
50_24,0.228632,7.615166,0.543775,0.543775,0.404510,1.831587,0.082943,0.033327,0.767750,0.722966,...,2.238344,-0.009766,0.000030,3.984393,-0.026213,1.0,0.146273,-0.000567,2.001063,0.000392
50_25,0.228632,8.232772,0.430527,0.430527,0.404510,6.258402,0.015655,0.026201,0.746783,0.723238,...,2.218349,-0.021701,0.000469,4.187977,0.045843,0.0,0.093430,0.007935,2.043632,0.000945
50_26,0.228632,6.549630,0.662450,0.662450,0.270641,5.168649,0.277326,0.029816,0.804893,0.685764,...,2.257441,-0.006975,0.000139,3.595821,0.279565,1.0,0.134507,0.005123,2.219708,0.000585
50_27,0.228632,7.367592,0.561258,0.561258,0.270641,2.824470,0.072055,0.041497,0.975342,0.755208,...,2.238223,0.010579,0.000120,3.641927,-0.043193,1.0,0.085082,0.009918,1.988028,0.000591


In [83]:
df_feat_sel.columns

Index(['smcac__fourier_entropy__bins_2', 'ae_spindle__cid_ce__normalize_True',
       'ae_spindle__partial_autocorrelation__lag_1',
       'ae_spindle__autocorrelation__lag_1', 'smcac__fourier_entropy__bins_10',
       'smcdc__fft_coefficient__attr_"abs"__coeff_1',
       'ae_spindle__autocorrelation__lag_2',
       'smcac__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'smcac__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',
       'smcac__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',
       ...
       'vib_table__binned_entropy__max_bins_10',
       'smcdc__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',
       'ae_table__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',
       'ae_spindle__fft_aggregated__aggtype_"centroid"',
       'ae_table__ar_coefficient__coeff_6__k_10',
       'vib_spindle__large_standard_deviation__r_0.2',
       'smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0',
       'ae_spindle__a